# Find duplicate questions on StackOverflow by their embeddings

In this assignment you will learn how to calculate a similarity for pieces of text. Using this approach you will know how to find duplicate questions from [StackOverflow](https://stackoverflow.com).

### Libraries

In this task you will you will need the following libraries:
- [StarSpace](https://github.com/facebookresearch/StarSpace) — a general-purpose model for efficient learning of entity embeddings from Facebook
- [Gensim](https://radimrehurek.com/gensim/) — a tool for solving various NLP-related tasks (topic modeling, text representation, ...)
- [Numpy](http://www.numpy.org) — a package for scientific computing.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
- [Nltk](http://www.nltk.org) — a platform to work with human language data.

### Data

The following cell will download all data required for this assignment into the folder `week3/data`.

In [2]:
import sys
sys.path.append("..")
from common.download_utils import download_week3_resources

download_week3_resources()

File data/train.tsv is already downloaded.
File data/validation.tsv is already downloaded.
File data/test.tsv is already downloaded.
File data/test_embeddings.tsv is already downloaded.


### Grading
We will create a grader instace below and use it to collect your answers. Note that these outputs will be stored locally inside grader and will be uploaded to platform only after running submiting function in the last part of this assignment. If you want to make partial submission, you can run that cell any time you want.

In [3]:
from grader import Grader

In [4]:
grader = Grader()

## Word embedding

To solve the problem, you will use two different models of embeddings:

 - [Pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google which were trained on a part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. You need to download it by following this [link](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing).
 - Representations using StarSpace on StackOverflow data sample. You will need to train them from scratch.

It's always easier to start with pre-trained embeddings. Unpack the pre-trained Goggle's vectors and upload them using the function [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) from gensim library with the parameter *binary=True*. If the size of the embeddings is larger than the avaliable memory, you could load only a part of the embeddings by defining the parameter *limit* (recommended: 500000).

In [5]:
import gensim
from gensim import models
help(models.KeyedVectors.load_word2vec_format)

Help on method load_word2vec_format in module gensim.models.keyedvectors:

load_word2vec_format(fname, fvocab=None, binary=False, encoding='utf8', unicode_errors='strict', limit=None, datatype=<class 'numpy.float32'>) method of builtins.type instance
    Load the input-hidden weight matrix from the original C word2vec-tool format.
    
    Note that the information stored in the file is incomplete (the binary tree is missing),
    so while you can query for word similarity etc., you cannot continue training
    with a model loaded this way.
    
    `binary` is a boolean indicating whether the data is in binary word2vec format.
    `norm_only` is a boolean indicating whether to only store normalised word2vec vectors in memory.
    Word counts are read from `fvocab` filename, if set (this is the file generated
    by `-save-vocab` flag of the original C tool).
    
    If you trained the C model using non-utf8 encoding for words, specify that
    encoding in `encoding`.
    
    `unicod

In [6]:
wv_embeddings = models.KeyedVectors.load_word2vec_format('./data/GoogleNews-vectors-negative300.bin',
                                                         binary=True, limit=500000)
######### YOUR CODE HERE #############

### How to work with Google's word2vec embeddings?

Once you have loaded the representations, make sure you can access them. First, you can check if the loaded embeddings contain a word:
    
    'word' in wv_embeddings
    
Second, to get the corresponding embedding you can use the square brackets:

    wv_embeddings['word']
 
### Checking that the embeddings are correct 
 
To prevent any errors during the first stage, we can check that the loaded embeddings are correct. You can call the function *check_embeddings*, implemented below, which runs 3 tests:
1. Find the most similar word for provided "positive" and "negative" words.
2. Find which word from the given list doesn’t go with the others.
3. Find the most similar word for the provided one.

In the right case the function will return the string *These embeddings look good*. Othervise, you need to validate the previous steps.

In [7]:
def check_embeddings(embeddings):
    error_text = "Something wrong with your embeddings ('%s test isn't correct)."
    most_similar = embeddings.most_similar(positive=['woman', 'king'], negative=['man'])
    if len(most_similar) < 1 or most_similar[0][0] != 'queen':
        return error_text % "Most similar"

    doesnt_match = embeddings.doesnt_match(['breakfast', 'cereal', 'dinner', 'lunch'])
    if doesnt_match != 'cereal':
        return error_text % "Doesn't match"
    
    most_similar_to_given = embeddings.most_similar_to_given('music', ['water', 'sound', 'backpack', 'mouse'])
    if most_similar_to_given != 'sound':
        return error_text % "Most similar to given"
    
    return "These embeddings look good."

In [8]:
print(check_embeddings(wv_embeddings))

These embeddings look good.


## From word to text embeddings

**Task 1 (Question2Vec).** Usually, we have word-based embeddings, but for the task we need to create a representation for the whole question. It could be done in different ways. In our case we will use a **mean** of all word vectors in the question. Now you need to implement the function *question_to_vec*, which calculates the question representation described above.

Note that there could be words without the corresponding embeddings. In this case, you can just skip these words and don't take them into account during calculating the result. If the question doesn't contain any known word with embedding, the function should return a zero vector.

In [9]:
import numpy as np

In [110]:
def question_to_vec(question, embeddings, dim=300):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    
    vec = np.zeros((dim,), dtype=np.float32)
    count = 0
    for w in question.split():
        if w in embeddings:
            count += 1
            vec += embeddings[w]
    if count == 0:
        return vec
    return vec/count
            
    ######################################
    ######### YOUR CODE HERE #############
    ######################################

To check the basic correctness of your implementation, run the function *question_to_vec_tests*.

In [14]:
def question_to_vec_tests():
    if (np.zeros(300) != question_to_vec('', wv_embeddings)).any():
        return "You need to return zero vector for empty question."
    if (np.zeros(300) != question_to_vec('thereisnosuchword', wv_embeddings)).any():
        return "You need to return zero vector for the question, which consists only unknown words."
    if (wv_embeddings['word'] != question_to_vec('word', wv_embeddings)).any():
        return "You need to check the corectness of your function."
    if ((wv_embeddings['I'] + wv_embeddings['am']) / 2 != question_to_vec('I am', wv_embeddings)).any():
        return "Your function should calculate a mean of word vectors."
    if (wv_embeddings['word'] != question_to_vec('thereisnosuchword word', wv_embeddings)).any():
        return "You should not consider words which embeddings are unknown."
    return "Basic tests are passed."

In [15]:
print(question_to_vec_tests())

Basic tests are passed.


You can submit embeddings for the questions from the file *test_embeddings.tsv* to earn the points. In this task you don't need to transform the text of a question somehow.

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
from util import array_to_string

In [19]:
question2vec_result = []
for question in open('data/test_embeddings.tsv'):
    question = question.strip()
    answer = question_to_vec(question, wv_embeddings)
    question2vec_result = np.append(question2vec_result, answer)

grader.submit_tag('Question2Vec', array_to_string(question2vec_result))

Current answer for task Question2Vec is: 0.019293891266
-0.0287272129208
0.0460561104119
0.0852593332529
0.0243055559695
-0.0729031041265
0.0...


Now we have a method to create a representation of any sentence and we are ready for the first evaluation. So, let's check how well our solution (Google's vectors + *question_to_vec*) will work.

## Evaluation of text similarity

We can imagine that if we use good embeddings, the cosine similarity between the duplicate sentences should be less than for the random ones. Overall, for each pair of duplicate sentences we can generate *R* random negative examples and find out the position of the correct duplicate.  

For example, we have the question *"Exceptions What really happens"* and we are sure that another question *"How does the catch keyword determine the type of exception that was thrown"* is a duplicate. But our model doesn't know it and tries to find out the best option also among questions like *"How Can I Make These Links Rotate in PHP"*, *"NSLog array description not memory address"* and *"PECL_HTTP not recognised php ubuntu"*. The goal of the model is to rank all these 4 questions (1 *positive* and *R* = 3 *negative*) in the way that the correct one is in the first place.

However, it is unnatural to count on that the best candidate will be always in the first place. So let us consider the place of the best candidate in the sorted list of candidates and formulate a metric based on it. We can fix some *K* — a reasonalble number of top-ranked elements and *N* — a number of queries (size of the sample).

### Hits@K

The first simple metric will be a number of correct hits for some *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)]$$

where $q_i$ is the i-th query, $dup_i$ is its duplicate, $topK(q_i)$ is the top K elements of the ranked sentences provided by our model and the operation $[dup_i \in topK(q_i)]$ equals 1 if the condition is true and 0 otherwise (more details about this operation could be found [here](https://en.wikipedia.org/wiki/Iverson_bracket)).


### DCG@K
The second one is a simplified [DCG metric](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K] $$

where $rank_{dup_i}$ is a position of the duplicate in the sorted list of the nearest sentences for the query $q_i$. According to this metric, the model gets a higher reward for a higher position of the correct answer. If the answer does not appear in topK at all, the reward is zero. 

### Evaluation examples

Let's calculate the described metrics for the toy example introduced above. In this case $N$ = 1 and the correct candidate for $q_1$ is *"How does the catch keyword determine the type of exception that was thrown"*. Consider the following ranking of the candidates:
1. *"How Can I Make These Links Rotate in PHP"*
2. *"How does the catch keyword determine the type of exception that was thrown"*
3. *"NSLog array description not memory address"*
4. *"PECL_HTTP not recognised php ubuntu"*

Using the ranking above, calculate *Hits@K* metric for *K = 1, 2, 4*: 
 
- [K = 1] $\text{Hits@1} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top1(q_i)] = [dup_1 \in top1(q_1)] = 0$ because the correct answer doesn't appear in the *top1* list.
- [K = 2] $\text{Hits@2} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top2(q_i)] = [dup_1 \in top2(q_1)] = 1$ because $rank_{dup_1} = 2$.
- [K = 4] $\text{Hits@4} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top4(q_i)] = [dup_1 \in top4(q_1)] = 1$

Using the ranking above, calculate *DCG@K* metric for *K = 1, 2, 4*:

- [K = 1] $\text{DCG@1} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = \frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = 0$ because the correct answer doesn't appear in the top1 list.
- [K = 2] $\text{DCG@2} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 2] = \frac{1}{\log_2{3}}$, because $rank_{dup_1} = 2$.
- [K = 4] $\text{DCG@4} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 4] = \frac{1}{\log_2{3}}$.


**Tasks 2 and 3 (HitsCount and DCGScore).** Implement the functions *hits_count* and *dcg_score* as described above. Each function has two arguments: *dup_ranks* and *k*. *dup_ranks* is a list which contains *values of ranks* of duplicates. For example, *dup_ranks* is *[2]* for the example provided above.

In [29]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list with ranks for each duplicate (the best rank is 1, the worst is len(dup_ranks))
        k: number of top-ranked elements

        result: float number
    """
    ######################################
    ######### YOUR CODE HERE #############
    count = 0
    for rank in dup_ranks:
        if rank <=k:
            count += 1
    return count/(len(dup_ranks)+1e-8)
    ######################################

Test your code on the tiny examples:

In [30]:
def test_hits():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1, 1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]
    
    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."

    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [31]:
print(test_hits())

Basic test are passed.


In [36]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list with ranks for each duplicate (the best rank is 1, the worst is len(dup_ranks))
        k: number of top-ranked elements

        result: float number
    """
    ######################################
    ######### YOUR CODE HERE #############
    score = 0
    for rank in dup_ranks:
        if rank <= k:
            score += 1/np.log2(1+rank)
    return score/(len(dup_ranks)+1e-8)
    ######################################

In [37]:
def test_dcg():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]

    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, (1 + (1 / (np.log2(3)))) / 2]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."
        
    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [38]:
print(test_dcg())

Basic test are passed.


Submit results of the functions *hits_count* and *dcg_score* for the following examples to earn the points.

In [39]:
test_examples = [
    [1],
    [1, 2],
    [2, 1],
    [1, 2, 3],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [9, 5, 4, 2, 8, 10, 7, 6, 1, 3],
    [4, 3, 5, 1, 9, 10, 7, 8, 2, 6],
    [5, 1, 7, 6, 2, 3, 8, 9, 10, 4],
    [6, 3, 1, 4, 7, 2, 9, 8, 10, 5],
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
]

In [40]:
hits_results = []
for example in test_examples:
    for k in range(len(example)):
        hits_results.append(hits_count(example, k + 1))
grader.submit_tag('HitsCount', array_to_string(hits_results))

Current answer for task HitsCount is: 0.9999999900000002
0.4999999975
0.999999995
0.4999999975
0.999999995
0.3333333322222222
0.6666666644...


In [41]:
dcg_results = []
for example in test_examples:
    for k in range(len(example)):
        dcg_results.append(dcg_score(example, k + 1))
grader.submit_tag('DCGScore', array_to_string(dcg_results))

Current answer for task DCGScore is: 0.99999999
0.4999999975
0.815464872708
0.4999999975
0.815464872708
0.333333332222
0.543643249378
0.7...


##  First solution: pre-trained embeddings

We will work with predefined train, validation and test corpora. All the files are tab-separated, but have a different format:
 - *train* corpus contains similar sentences at the same row.
 - *validation* corpus contains the following columns: *question*, *similar question*, *negative example 1*, *negative example 2*, ... 
 - *test* corpus contains the following columns: *question*, *example 1*, *example 2*, ...

Validation corpus will be used for the intermediate validation of models. The test data will be necessary for submitting the quality of your model in the system.

Now you should upload *validation* corpus to evaluate current solution.

In [72]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [73]:
validation = read_corpus('data/validation.tsv')
######### YOUR CODE HERE #############

In [74]:
from sklearn.metrics.pairwise import cosine_similarity

We will use cosine distance to rank candidate questions which you need to implement in the function *rank_candidates*. The function should return a sorted list of pairs *(initial position in candidates list, candidate)*. Index of some pair corresponds to its rank (the first is the best). For example, if the list of candidates was *[a, b, c]* and the most similar is *c*, then *a* and *b*, the function should return a list *[(2, c), (0, a), (1, b)]*.

Pay attention, if you use the function *cosine_similarity* from *sklearn.metrics.pairwise* to calculate similarity because it works in a different way: most similar objects has greatest similarity. It's preferable to use a vectorized version of *cosine_similarity* function. Try to compute similarity at once and not use list comprehension. It should speed up your computations significantly.

In [75]:
def rank_candidates(question, candidates, embeddings, dim=300):
    """
        question: a string
        candidates: a list of strings (candidates) which we want to rank
        embeddings: some embeddings
        dim: dimension of the current embeddings
        
        result: a list of pairs (initial position in the list, question)
    """
    # print("ffffffff")
    ######################################
    ######### YOUR CODE HERE #############
    vecq = np.array([np.array(question_to_vec(question, embeddings, dim))])
    vecc = np.array([np.array(question_to_vec(can, embeddings, dim)) for can in candidates])
    scores = list(cosine_similarity(vecq, vecc)[0])
    tl = [(i, candidates[i], scores[i]) for i in range(len(candidates))]
    stl = sorted(tl, key=lambda x:x[2], reverse=True)
    return [(t[0],t[1]) for t in stl]
    ######################################

Test your code on the tiny examples:

In [76]:
def test_rank_candidates():
    questions = ['converting string to list', 'Sending array via Ajax fails']
    candidates = [['Convert Google results object (pure js) to Python object', 
                   'C# create cookie from string and send it',
                   'How to use jQuery AJAX for an outside domain?'], 
                  ['Getting all list items of an unordered list in PHP', 
                   'WPF- How to update the changes in list item of a list', 
                   'select2 not displaying search results']]
    results = [[(1, 'C# create cookie from string and send it'), 
                (0, 'Convert Google results object (pure js) to Python object'), 
                (2, 'How to use jQuery AJAX for an outside domain?')],
               [(0, 'Getting all list items of an unordered list in PHP'), 
                (2, 'select2 not displaying search results'), 
                (1, 'WPF- How to update the changes in list item of a list')]]
    for question, q_candidates, result in zip(questions, candidates, results):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, 300)
        if not np.all(ranks == result):
            return "Check the function."
    return "Basic tests are passed."

In [77]:
print(test_rank_candidates())

Basic tests are passed.


Now we can test the quality of the current approach. Run the next two cells to get the results. Pay attention that calculation of similarity between vectors takes time and this calculation is computed approximately in 10 minutes.

In [78]:
wv_ranking = []
for line in validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [79]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.212 | Hits@   1: 0.212
DCG@   5: 0.267 | Hits@   5: 0.315
DCG@  10: 0.282 | Hits@  10: 0.363
DCG@ 100: 0.320 | Hits@ 100: 0.552
DCG@ 500: 0.353 | Hits@ 500: 0.811
DCG@1000: 0.373 | Hits@1000: 1.000


If you did all the steps correctly, you should be frustrated by the received results. Let's try to understand why the quality is so low. First of all, when you work with some data it is necessary to have an idea how the data looks like. Print several questions from the data:

In [80]:
for line in validation[:3]:
    q, *examples = line
    print(q, *examples[:3])

How to print a binary heap tree without recursion? How do you best convert a recursive function to an iterative one? How can i use ng-model with directive in angular js flash: drawing and erasing
How to start PhoneStateListener programmatically? PhoneStateListener and service Java cast object[] to model WCF and What does this mean?
jQuery: Show a div2 when mousenter over div1 is over when hover on div1 depenting on if it is on div2 or not it should act differently How to run selenium in google app engine/cloud? Python Comparing two lists of strings for similarities


As you can see, we deal with the raw data. It means that we have many punctuation marks, special characters and unlowercased letters. In our case, it could lead to the situation where we can't find some embeddings, e.g. for the word "grid?". 

To solve this problem you should use the functions *text_prepare* from the previous assignments to prepare the data.

In [81]:
from util import text_prepare

Now transform all the questions from the validation set:

In [85]:
prepared_validation = []
for line in validation:
    q, *ex = line
    q = text_prepare(q)
    for i,e in enumerate(ex):
        ex[i] = text_prepare(e)
    prepared_validation.append([q,*ex])
    ######### YOUR CODE HERE #############

Let's evaluate the approach again after the preparation:

In [86]:
wv_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [87]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_prepared_ranking, k), 
                                              k, hits_count(wv_prepared_ranking, k)))

DCG@   1: 0.310 | Hits@   1: 0.310
DCG@   5: 0.380 | Hits@   5: 0.443
DCG@  10: 0.397 | Hits@  10: 0.494
DCG@ 100: 0.430 | Hits@ 100: 0.661
DCG@ 500: 0.453 | Hits@ 500: 0.835
DCG@1000: 0.470 | Hits@1000: 1.000


Now, prepare also train and test data, because you will need it in the future:

In [90]:
def prepare_file(in_, out_):
    out = open(out_, 'w')
    for line in open(in_, encoding='utf8'):
        line = line.strip().split('\t')
        new_line = [text_prepare(q) for q in line]
        print(*new_line, sep='\t', file=out)
    out.close()

In [91]:
######################################
######### YOUR CODE HERE #############
######################################
prepare_file('./data/validation.tsv', './data/tp_v.tsv')
prepare_file('./data/test.tsv', './data/tp_t.tsv')

In [100]:
prepare_file('./data/train.tsv', './data/tp_train.tsv')

**Task 4 (W2VTokenizedRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates to earn the points. The calculations should take about 3-5 minutes. Pay attention that the function *rank_candidates* returns a ranking, while in this case you should find a position in this ranking. Ranks should start with 1.

In [92]:
from util import matrix_to_string

In [93]:
w2v_ranks_results = []
prepared_test_data = './data/tp_t.tsv' ######### YOUR CODE HERE #############
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, wv_embeddings, 300)
    ranked_candidates = [r[0] for r in ranks]
    w2v_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('W2VTokenizedRanks', matrix_to_string(w2v_ranks_results))

Current answer for task W2VTokenizedRanks is: 95	94	7	9	64	36	31	93	23	100	99	20	60	6	97	48	70	37	41	96	29	56	2	65	68	44	27	25	57	62	11	87	50	66	7...


## Advanced solution: StarSpace embeddings

Now you are ready to train your own word embeddings! In particular, you need to train embeddings specially for our task of duplicates detection. Unfortunately, StarSpace could not be run on Windows and we recommend to use provided
docker container or other alternatives. Don't delete results of this task because you will need it in the final project.

### How it works and what's the main difference with word2vec?
The main point in this section is that StarSpace can be trained specifically for some tasks. In contrast to word2vec model, which tries to train similar embeddings for words in similar contexts, StarSpace uses embeddings for the whole sentence (just as a sum of embeddings of words and phrases). Despite the fact that in both cases we get word embeddings as a result of the training, StarSpace embeddings are trained using some supervised data, e.g. a set of similar sentence pairs, and thus they can better suit the task.

In our case, StarSpace should use two types of sentence pairs for training: "positive" and "negative". "Positive" examples are extracted from the train sample (duplicates, high similarity) and the "negative" examples are generated randomly (low similarity assumed). 

### How to choose the best params for the model?
Normally, you would start with some default choice and then run extensive experiments to compare different strategies. However, we have some recommendations ready for you to save your time:
- Be careful with choosing the suitable training mode. In this task we want to explore texts similarity which corresponds to *trainMode = 3*.
- Use adagrad optimization (parameter *adagrad = true*).
- Set the length of phrase equal to 1 (parameter *ngrams*), because we need embeddings only for words.
- Don't use a large number of *epochs* (we think that 5 should be enough).
- Try dimension *dim* equal to 100.
- To compare embeddings usually *cosine* *similarity* is used.
- Set *minCount* greater than 1 (for example, 2) if you don't want to get embeddings for extremely rare words.
- Parameter *verbose = true* could show you the progress of the training process.
- Set parameter *fileFormat* equals *labelDoc*.
- Parameter *negSearchLimit* is responsible for a nuber of negative examples which is used during the training. We think that 10 will be enought for this task.
- To increase a speed of training we recommend to set *learning rate* to 0.05.

Train StarSpace embeddings for unigrams on the train dataset. You don't need to change the format of the input data. Just don't forget to use prepared version of the training data. 

If you follow the instruction, the training process will take about 1 hour.

In [105]:
######### TRAINING HAPPENING HERE #############
!Starspace/starspace train -trainFile './data/tp_train.tsv' -model "sodd" -trainMode 3 \
    -adagrad true -ngrams 1 -epoch 5 -dim 100 -minCount 2 -verbose true -fileFormat "labelDoc" \
    -negSearchLimit 10 -similarity "cosine" -lr 0.05

Arguments: 
lr: 0.05
dim: 100
epoch: 5
maxTrainTime: 8640000
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
thread: 10
minCount: 2
minCountLabel: 1
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 3
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
Start to initialize starspace model.
Build dict from input file : ./data/tp_train.tsv
Read 12M words
Number of words in dictionary:  95058
Number of labels in dictionary: 0
Loading data from file : ./data/tp_train.tsv
Total number of examples loaded : 999740
Initialized model weights. Model size :
matrix : 95058 100
Training epoch 0: 0.05 0.01


Epoch: 73.4%  lr: 0.042993  loss: 0.009974  eta: 0h8m  tot: 0h1m28s  (14.7%)0%  lr: 0.049940  loss: 0.047360  eta: 0h13m  tot: 0h0m1s  (0.2%)%  lr: 0.049920  loss: 0.044684  eta: 0h13m  tot: 0h0m2s  (0.3%)2.1%  lr: 0.049870  loss: 0.037615  eta: 0h11m  tot: 0h0m2s  (0.4%)3.5%  lr: 0.049720  loss: 0.031830  eta: 0h10m  tot: 0h0m4s  (0.7%)3.6%  lr: 0.049710  loss: 0.031552  eta: 0h10m  tot: 0h0m4s  (0.7%)4.3%  lr: 0.049600  loss: 0.029458  eta: 0h10m  tot: 0h0m5s  (0.9%)4.4%  lr: 0.049600  loss: 0.029193  eta: 0h10m  tot: 0h0m5s  (0.9%)4.9%  lr: 0.049580  loss: 0.028103  eta: 0h10m  tot: 0h0m6s  (1.0%)5.0%  lr: 0.049550  loss: 0.027648  eta: 0h10m  tot: 0h0m6s  (1.0%)6.0%  lr: 0.049449  loss: 0.025782  eta: 0h10m  tot: 0h0m7s  (1.2%)7.7%  lr: 0.049259  loss: 0.023623  eta: 0h10m  tot: 0h0m9s  (1.5%)8.3%  lr: 0.049229  loss: 0.022892  eta: 0h10m  tot: 0h0m10s  (1.7%)8.4%  lr: 0.049219  loss: 0.022744  eta: 0h10m  tot: 0h0m10s  (1.7%)%  lr: 0.049019  loss: 0.021411  eta: 0h10m  tot: 0h0m12

Epoch: 100.0%  lr: 0.040250  loss: 0.008897  eta: 0h8m  tot: 0h2m0s  (20.0%)74.1%  lr: 0.042933  loss: 0.009931  eta: 0h8m  tot: 0h1m29s  (14.8%)  eta: 0h8m  tot: 0h1m30s  (14.9%)75.1%  lr: 0.042833  loss: 0.009894  eta: 0h8m  tot: 0h1m31s  (15.0%)m31s  (15.1%)%  lr: 0.042653  loss: 0.009821  eta: 0h8m  tot: 0h1m33s  (15.4%)77.4%  lr: 0.042613  loss: 0.009797  eta: 0h8m  tot: 0h1m34s  (15.5%)77.8%  lr: 0.042603  loss: 0.009784  eta: 0h8m  tot: 0h1m34s  (15.6%)77.9%  lr: 0.042603  loss: 0.009783  eta: 0h8m  tot: 0h1m34s  (15.6%)78.2%  lr: 0.042543  loss: 0.009772  eta: 0h8m  tot: 0h1m35s  (15.6%)78.3%  lr: 0.042533  loss: 0.009772  eta: 0h8m  tot: 0h1m35s  (15.7%)79.6%  lr: 0.042463  loss: 0.009716  eta: 0h8m  tot: 0h1m36s  (15.9%)80.0%  lr: 0.042433  loss: 0.009691  eta: 0h8m  tot: 0h1m37s  (16.0%)80.5%  lr: 0.042383  loss: 0.009674  eta: 0h8m  tot: 0h1m37s  (16.1%)80.7%  lr: 0.042362  loss: 0.009664  eta: 0h8m  tot: 0h1m38s  (16.1%)80.9%  lr: 0.042342  loss: 0.009654  eta: 0h8m  tot: 

Epoch: 57.5%  lr: 0.034515  loss: 0.002708  eta: 0h6m  tot: 0h3m10s  (31.5%)7%  lr: 0.039920  loss: 0.003003  eta: 0h6m  tot: 0h2m5s  (20.1%)0.9%  lr: 0.039880  loss: 0.002698  eta: 0h6m  tot: 0h2m5s  (20.2%)1.0%  lr: 0.039880  loss: 0.002523  eta: 0h7m  tot: 0h2m5s  (20.2%)1.1%  lr: 0.039860  loss: 0.002405  eta: 0h7m  tot: 0h2m5s  (20.2%)2.3%  lr: 0.039760  loss: 0.002445  eta: 0h7m  tot: 0h2m7s  (20.5%)%  lr: 0.039720  loss: 0.002411  eta: 0h7m  tot: 0h2m7s  (20.5%)2.9%  lr: 0.039710  loss: 0.002455  eta: 0h7m  tot: 0h2m7s  (20.6%)3.8%  lr: 0.039590  loss: 0.002354  eta: 0h7m  tot: 0h2m8s  (20.8%)4.4%  lr: 0.039500  loss: 0.002315  eta: 0h7m  tot: 0h2m9s  (20.9%)4.7%  lr: 0.039489  loss: 0.002289  eta: 0h7m  tot: 0h2m9s  (20.9%)5.0%  lr: 0.039449  loss: 0.002306  eta: 0h7m  tot: 0h2m10s  (21.0%)5.7%  lr: 0.039439  loss: 0.002304  eta: 0h7m  tot: 0h2m10s  (21.1%)6.0%  lr: 0.039379  loss: 0.002311  eta: 0h7m  tot: 0h2m11s  (21.2%)6.3%  lr: 0.039339  loss: 0.002345  eta: 0h7m  tot: 0h2

Epoch: 100.0%  lr: 0.030431  loss: 0.002693  eta: 0h5m  tot: 0h4m0s  (40.0%)%  lr: 0.034455  loss: 0.002709  eta: 0h6m  tot: 0h3m11s  (31.6%)58.9%  lr: 0.034354  loss: 0.002706  eta: 0h6m  tot: 0h3m12s  (31.8%)31.8%)59.3%  lr: 0.034304  loss: 0.002705  eta: 0h6m  tot: 0h3m13s  (31.9%)59.8%  lr: 0.034274  loss: 0.002703  eta: 0h6m  tot: 0h3m13s  (32.0%)61.0%  lr: 0.034164  loss: 0.002701  eta: 0h6m  tot: 0h3m14s  (32.2%)61.8%  lr: 0.034074  loss: 0.002690  eta: 0h6m  tot: 0h3m16s  (32.4%)62.3%  lr: 0.034024  loss: 0.002693  eta: 0h6m  tot: 0h3m16s  (32.5%)62.7%  lr: 0.033984  loss: 0.002695  eta: 0h6m  tot: 0h3m17s  (32.5%)62.9%  lr: 0.033954  loss: 0.002693  eta: 0h6m  tot: 0h3m17s  (32.6%)63.2%  lr: 0.033934  loss: 0.002698  eta: 0h6m  tot: 0h3m17s  (32.6%)%  lr: 0.033804  loss: 0.002704  eta: 0h6m  tot: 0h3m18s  (32.8%)64.1%  lr: 0.033754  loss: 0.002699  eta: 0h6m  tot: 0h3m18s  (32.8%)64.4%  lr: 0.033724  loss: 0.002693  eta: 0h6m  tot: 0h3m18s  (32.9%)64.5%  lr: 0.033724  loss: 0.

...

Epoch: 100.0%  lr: 0.020371  loss: 0.001939  eta: 0h3m  tot: 0h5m59s  (60.0%)9.0%  lr: 0.023393  loss: 0.001958  eta: 0h4m  tot: 0h5m24s  (53.8%)69.9%  lr: 0.023303  loss: 0.001963  eta: 0h4m  tot: 0h5m25s  (54.0%)70.2%  lr: 0.023283  loss: 0.001962  eta: 0h4m  tot: 0h5m26s  (54.0%)71.0%  lr: 0.023153  loss: 0.001960  eta: 0h4m  tot: 0h5m26s  (54.2%)71.3%  lr: 0.023123  loss: 0.001958  eta: 0h4m  tot: 0h5m27s  (54.3%)75.0%  lr: 0.022733  loss: 0.001960  eta: 0h4m  tot: 0h5m31s  (55.0%)75.7%  lr: 0.022653  loss: 0.001963  eta: 0h4m  tot: 0h5m32s  (55.1%)75.9%  lr: 0.022633  loss: 0.001963  eta: 0h4m  tot: 0h5m32s  (55.2%)76.2%  lr: 0.022613  loss: 0.001963  eta: 0h4m  tot: 0h5m32s  (55.2%)76.7%  lr: 0.022603  loss: 0.001962  eta: 0h4m  tot: 0h5m33s  (55.3%)77.1%  lr: 0.022543  loss: 0.001961  eta: 0h4m  tot: 0h5m33s  (55.4%)77.7%  lr: 0.022473  loss: 0.001961  eta: 0h4m  tot: 0h5m34s  (55.5%)78.4%  lr: 0.022382  loss: 0.001960  eta: 0h4m  tot: 0h5m35s  (55.7%)78.5%  lr: 0.022372  loss: 

Epoch: 54.8%  lr: 0.014885  loss: 0.001518  eta: 0h2m  tot: 0h7m4s  (71.0%))2%  lr: 0.019970  loss: 0.001468  eta: 0h6m  tot: 0h6m4s  (60.0%)0.3%  lr: 0.019950  loss: 0.001341  eta: 0h5m  tot: 0h6m4s  (60.1%)0.4%  lr: 0.019930  loss: 0.001523  eta: 0h5m  tot: 0h6m4s  (60.1%)0.7%  lr: 0.019910  loss: 0.001693  eta: 0h4m  tot: 0h6m5s  (60.1%)1.0%  lr: 0.019890  loss: 0.001739  eta: 0h4m  tot: 0h6m5s  (60.2%)1.1%  lr: 0.019890  loss: 0.001743  eta: 0h4m  tot: 0h6m5s  (60.2%)1.3%  lr: 0.019880  loss: 0.001839  eta: 0h4m  tot: 0h6m5s  (60.3%)2.7%  lr: 0.019750  loss: 0.001704  eta: 0h4m  tot: 0h6m7s  (60.5%)3.0%  lr: 0.019720  loss: 0.001696  eta: 0h4m  tot: 0h6m7s  (60.6%)3.1%  lr: 0.019710  loss: 0.001660  eta: 0h4m  tot: 0h6m7s  (60.6%)3.3%  lr: 0.019700  loss: 0.001663  eta: 0h4m  tot: 0h6m8s  (60.7%)3.5%  lr: 0.019700  loss: 0.001632  eta: 0h3m  tot: 0h6m8s  (60.7%)3.9%  lr: 0.019670  loss: 0.001631  eta: 0h3m  tot: 0h6m8s  (60.8%)4.1%  lr: 0.019670  loss: 0.001637  eta: 0h3m  tot: 0h6

Epoch: 100.0%  lr: 0.010391  loss: 0.001552  eta: 0h1m  tot: 0h7m57s  (80.0%).5%  lr: 0.014785  loss: 0.001527  eta: 0h2m  tot: 0h7m5s  (71.1%)55.7%  lr: 0.014775  loss: 0.001528  eta: 0h2m  tot: 0h7m5s  (71.1%)55.9%  lr: 0.014715  loss: 0.001530  eta: 0h2m  tot: 0h7m6s  (71.2%)56.4%  lr: 0.014645  loss: 0.001529  eta: 0h2m  tot: 0h7m6s  (71.3%)57.6%  lr: 0.014545  loss: 0.001540  eta: 0h2m  tot: 0h7m7s  (71.5%)57.7%  lr: 0.014545  loss: 0.001540  eta: 0h2m  tot: 0h7m8s  (71.5%)58.0%  lr: 0.014525  loss: 0.001538  eta: 0h2m  tot: 0h7m8s  (71.6%)58.1%  lr: 0.014525  loss: 0.001539  eta: 0h2m  tot: 0h7m8s  (71.6%)58.2%  lr: 0.014525  loss: 0.001540  eta: 0h2m  tot: 0h7m8s  (71.6%)58.5%  lr: 0.014485  loss: 0.001539  eta: 0h2m  tot: 0h7m9s  (71.7%)58.7%  lr: 0.014475  loss: 0.001537  eta: 0h2m  tot: 0h7m9s  (71.7%)59.1%  lr: 0.014445  loss: 0.001536  eta: 0h2m  tot: 0h7m9s  (71.8%)59.6%  lr: 0.014425  loss: 0.001541  eta: 0h2m  tot: 0h7m10s  (71.9%)60.9%  lr: 0.014294  loss: 0.001541  eta

Epoch: 65.1%  lr: 0.003924  loss: 0.001361  eta: <1min   tot: 0h9m15s  (93.0%)4%  lr: 0.009960  loss: 0.001533  eta: 0h1m  tot: 0h8m2s  (80.1%)0.6%  lr: 0.009950  loss: 0.001303  eta: 0h1m  tot: 0h8m2s  (80.1%)1.2%  lr: 0.009900  loss: 0.001273  eta: 0h2m  tot: 0h8m3s  (80.2%)1.3%  lr: 0.009890  loss: 0.001155  eta: 0h2m  tot: 0h8m3s  (80.3%)1.9%  lr: 0.009800  loss: 0.001159  eta: 0h2m  tot: 0h8m4s  (80.4%)2.2%  lr: 0.009780  loss: 0.001235  eta: 0h1m  tot: 0h8m4s  (80.4%)3.2%  lr: 0.009660  loss: 0.001266  eta: 0h1m  tot: 0h8m5s  (80.6%)4.1%  lr: 0.009620  loss: 0.001310  eta: 0h1m  tot: 0h8m6s  (80.8%)4.2%  lr: 0.009610  loss: 0.001295  eta: 0h1m  tot: 0h8m7s  (80.8%)4.5%  lr: 0.009590  loss: 0.001301  eta: 0h1m  tot: 0h8m7s  (80.9%)4.7%  lr: 0.009580  loss: 0.001288  eta: 0h1m  tot: 0h8m7s  (80.9%)4.8%  lr: 0.009550  loss: 0.001280  eta: 0h1m  tot: 0h8m7s  (81.0%)5.2%  lr: 0.009479  loss: 0.001271  eta: 0h1m  tot: 0h8m8s  (81.0%)5.9%  lr: 0.009449  loss: 0.001280  eta: 0h1m  tot: 0

Epoch: 99.9%  lr: 0.000310  loss: 0.001364  eta: <1min   tot: 0h9m55s  (100.0%)5.4%  lr: 0.003894  loss: 0.001360  eta: <1min   tot: 0h9m15s  (93.1%)65.4%  lr: 0.003894  loss: 0.001360  eta: <1min   tot: 0h9m15s  (93.1%)66.0%  lr: 0.003824  loss: 0.001359  eta: <1min   tot: 0h9m16s  (93.2%)66.7%  lr: 0.003734  loss: 0.001359  eta: <1min   tot: 0h9m17s  (93.3%)%  lr: 0.003734  loss: 0.001360  eta: <1min   tot: 0h9m17s  (93.4%)67.6%  lr: 0.003644  loss: 0.001361  eta: <1min   tot: 0h9m18s  (93.5%)68.8%  lr: 0.003544  loss: 0.001360  eta: <1min   tot: 0h9m19s  (93.8%)68.9%  lr: 0.003544  loss: 0.001360  eta: <1min   tot: 0h9m19s  (93.8%)69.4%  lr: 0.003504  loss: 0.001356  eta: <1min   tot: 0h9m20s  (93.9%)69.8%  lr: 0.003434  loss: 0.001360  eta: <1min   tot: 0h9m21s  (94.0%)70.5%  lr: 0.003303  loss: 0.001356  eta: <1min   tot: 0h9m22s  (94.1%)71.7%  lr: 0.003113  loss: 0.001354  eta: <1min   tot: 0h9m23s  (94.3%)72.0%  lr: 0.003093  loss: 0.001353  eta: <1min   tot: 0h9m23s  (94.4%)72.

And now we can compare the new embeddings with the previous ones. You can find trained word vectors in the file *[model_file_name].tsv*. Upload the embeddings from StarSpace into a dict. 

In [108]:
starspace_embeddings = {}
for line in open('./sodd.tsv'):
    word,*vec = line.strip().split()
    vf = []
    for v in vec:
        # print(v)
        vf.append(float(v))
    starspace_embeddings[word] = np.array(vf)
######### YOUR CODE HERE #############

In [111]:
ss_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ss_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [112]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking, k), 
                                               k, hits_count(ss_prepared_ranking, k)))

DCG@   1: 0.521 | Hits@   1: 0.521
DCG@   5: 0.615 | Hits@   5: 0.697
DCG@  10: 0.634 | Hits@  10: 0.754
DCG@ 100: 0.666 | Hits@ 100: 0.906
DCG@ 500: 0.675 | Hits@ 500: 0.981
DCG@1000: 0.678 | Hits@1000: 1.000


Due to training for the particular task with the supervised data, you should expect to obtain a higher quality than for the previous approach. In additiion, despite the fact that StarSpace's trained vectors have a smaller dimension than word2vec's, it provides better results in this task.

**Task 5 (StarSpaceRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates for trained representation.

In [113]:
starspace_ranks_results = []
prepared_test_data = './data/tp_t.tsv' ######### YOUR CODE HERE #############
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ranked_candidates = [r[0] for r in ranks]
    starspace_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('StarSpaceRanks', matrix_to_string(starspace_ranks_results))

Current answer for task StarSpaceRanks is: 91	11	68	53	25	83	34	84	95	79	24	37	39	48	96	86	12	42	47	98	19	28	8	78	100	62	97	49	69	5	71	59	82	29...


### Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. <b>Note:</b> Token expires 30 minutes after generation.

In [114]:
STUDENT_EMAIL = 'felixdae@gmail.com'# EMAIL 
STUDENT_TOKEN = 'IC50UU7wk9gGC79j'# TOKEN 
grader.status()

You want to submit these parts:
Task Question2Vec: 0.019293891266
-0.0287272129208
0.0460561104119
0.0852593332529
0.0243055559695
-0.0729031041265
0.0...
Task HitsCount: 0.9999999900000002
0.4999999975
0.999999995
0.4999999975
0.999999995
0.3333333322222222
0.6666666644...
Task DCGScore: 0.99999999
0.4999999975
0.815464872708
0.4999999975
0.815464872708
0.333333332222
0.543643249378
0.7...
Task W2VTokenizedRanks: 95	94	7	9	64	36	31	93	23	100	99	20	60	6	97	48	70	37	41	96	29	56	2	65	68	44	27	25	57	62	11	87	50	66	7...
Task StarSpaceRanks: 91	11	68	53	25	83	34	84	95	79	24	37	39	48	96	86	12	42	47	98	19	28	8	78	100	62	97	49	69	5	71	59	82	29...


If you want to submit these answers, run cell below

In [115]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
